# Next Steps


1. Rewrite optModel and optDataset module
2. Write a training loop
3. Define regret and track regret/runtime
4. Plot

## Test optModel

In [7]:
import cvxpy as cp
import numpy as np
import warnings
import sys
from IPython.core.interactiveshell import InteractiveShell
import torch
import torch.optim as optim
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import matplotlib.pyplot as plt

np.random.seed(42)
torch.manual_seed(42)

warnings.filterwarnings("ignore")
InteractiveShell.ast_node_interactivity = "all"
sys.path.insert(1,'E:\\User\\Stevens\\Spring 2024\\PTO - Fairness\\myGit\\myUtils')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [8]:
class optModel:
    def __init__(self, x, r, c, Q, alpha):
        self.alpha = alpha
        self.Q = Q
        self.r = r
        self.c = c
        self.x = x
        self.num_data, self.num_items, self.num_features = x.shape

    def setObj(self, r, c):
        if self.alpha == 1:
            self.objective = cp.sum(cp.log(cp.multiply(r, self.d)))
        else:
            self.objective = cp.sum(cp.power(cp.multiply(r, self.d), 1 - self.alpha)) / (1 - self.alpha)
        
        self.constraints = [
            self.d >= 0,
            cp.sum(cp.multiply(c, self.d)) <= self.Q
        ]
        self.problem = cp.Problem(cp.Maximize(self.objective), self.constraints)

    def solve(self, closed=False):
        opt_sol = []
        opt_val = []
        if closed:
            return self.solveC()

        for i in range(self.num_data):
            self.d = cp.Variable(self.num_items)
            self.setObj(self.r[i], self.c[i])
            self.problem.solve(abstol=1e-9, reltol=1e-9, feastol=1e-9)
            opt_sol.append(self.d.value.reshape(1, self.num_items))
            opt_val.append(self.problem.value)

        opt_sol = np.concatenate(opt_sol)
        return opt_sol, opt_val

    def solveC(self):
        if self.alpha == 1:
            return "Work in progress"
        
        opt_sols_c = []
        opt_vals_c = []
        for i in range(self.num_data):
            S = np.sum(self.c[i] ** (1 - 1 / self.alpha) * self.r[i] ** (-1 + 1 / self.alpha))
            opt_sol_c = (self.c[i] ** (-1 / self.alpha) * self.r[i] ** (-1 + 1 / self.alpha) * self.Q) / S
            opt_val_c = np.sum((self.r[i] * opt_sol_c) ** (1 - self.alpha)) / (1 - self.alpha)
            opt_sols_c.append(opt_sol_c)
            opt_vals_c.append(opt_val_c)
        
        opt_sols_c = np.array(opt_sols_c)
        return opt_sols_c, opt_vals_c

## 生成数据 

In [9]:
def genData(num_data, num_features, num_items, seed=42, Q=100, dim=1, deg=1, noise_width=0.5, epsilon=0.1):
    rnd = np.random.RandomState(seed)
    n = num_data
    p = num_features
    m = num_items
    
    x = rnd.normal(0, 1, (n, m, p))
    B = rnd.binomial(1, 0.5, (m, p))

    c = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            values = (np.dot(B[j], x[i, j].reshape(p, 1)).T / np.sqrt(p) + 3) ** deg + 1
            values *= 5
            values /= 3.5 ** deg
            epislon = rnd.uniform(1 - noise_width, 1 + noise_width, 1)
            values *= epislon
            values = np.ceil(values)
            c[i, j] = values

    c = c.astype(np.float64)
    r = rnd.normal(0, 1, (n, m))
    r = 1 / (1 + np.exp(-r))

    return x, r, c, Q

In [10]:
# Test optModel with synthetic data
x, r, c, Q = genData(2000, 20, 10)
alpha = 0.5

# Create an instance of the optModel class
optmodel = optModel(x, r, c, Q, alpha)

# Solve the optimization problem
opt_sol, opt_val = optmodel.solve()

print("Optimal solution:", opt_sol[0])
print("Objective value:", opt_val[0])

opt_sol_c, opt_val_c = optmodel.solve(closed=True)

print("Optimal solution (closed form):", opt_sol_c[0])
print("Objective value (closed form):", opt_val_c[0])

# Are they the same?
np.allclose(opt_sol, opt_sol_c, atol=1e-4, rtol=1e-4)
np.allclose(opt_val, opt_val_c)

Optimal solution: [1.9280332  4.93743494 0.7083329  2.73212343 3.71207079 1.47603921
 0.35599286 0.37031601 0.32396207 1.71644314]
Objective value: 15.199442184027435
Optimal solution (closed form): [1.92803807 4.93741623 0.70833192 2.7321237  3.71205201 1.47604698
 0.35599551 0.37031548 0.32396474 1.71644999]
Objective value (closed form): 15.199442184254428


True

True

## 测试optDataset

In [11]:
class optDataset(Dataset):
    def __init__(self, x, r, c, Q, alpha, closed=False):
        self.closed = closed
        self.x = x
        self.r = r
        self.c = c
        self.Q = Q
        self.alpha = alpha
        self.num_data, self.num_items, self.num_features = x.shape

        self._solve_optimization_problems()

    def _solve_optimization_problems(self):
        optmodel = optModel(self.x, self.r, self.c, self.Q, self.alpha)
        self.opt_sols, self.opt_vals = optmodel.solve()
        self.opt_sols_c, self.opt_vals_c = optmodel.solve(closed=True)
        self.opt_vals = np.array(self.opt_vals)
        self.opt_vals_c = np.array(self.opt_vals_c)

    def __len__(self):
        return len(self.r)

    def __getitem__(self, idx):
        ret = (
                torch.tensor(self.x[idx]),
                torch.tensor(self.r[idx]),
                torch.tensor(self.c[idx]),
                torch.tensor(self.opt_sols[idx]),
                torch.tensor(self.opt_vals[idx]),
                
            )
        if self.closed:
            ret = ret + (torch.tensor(self.opt_sols_c[idx]),
                torch.tensor(self.opt_vals_c[idx]),
    )                
        return ret



In [12]:
# Test optDatasetRd
dataset = optDataset(x, r, c, Q, alpha)
dataset[0]

(tensor([[ 0.4967, -0.1383,  0.6477,  1.5230, -0.2342, -0.2341,  1.5792,  0.7674,
          -0.4695,  0.5426, -0.4634, -0.4657,  0.2420, -1.9133, -1.7249, -0.5623,
          -1.0128,  0.3142, -0.9080, -1.4123],
         [ 1.4656, -0.2258,  0.0675, -1.4247, -0.5444,  0.1109, -1.1510,  0.3757,
          -0.6006, -0.2917, -0.6017,  1.8523, -0.0135, -1.0577,  0.8225, -1.2208,
           0.2089, -1.9597, -1.3282,  0.1969],
         [ 0.7385,  0.1714, -0.1156, -0.3011, -1.4785, -0.7198, -0.4606,  1.0571,
           0.3436, -1.7630,  0.3241, -0.3851, -0.6769,  0.6117,  1.0310,  0.9313,
          -0.8392, -0.3092,  0.3313,  0.9755],
         [-0.4792, -0.1857, -1.1063, -1.1962,  0.8125,  1.3562, -0.0720,  1.0035,
           0.3616, -0.6451,  0.3614,  1.5380, -0.0358,  1.5646, -2.6197,  0.8219,
           0.0870, -0.2990,  0.0918, -1.9876],
         [-0.2197,  0.3571,  1.4779, -0.5183, -0.8085, -0.5018,  0.9154,  0.3288,
          -0.5298,  0.5133,  0.0971,  0.9686, -0.7021, -0.3277, -0.3921, -

In [13]:
num_items = x.shape[1]
num_features = x.shape[2]
print("Number of items:", num_items)
print("Number of features:", num_features)

class LogisticRegressionModel(nn.Module):
    def __init__(self, num_items, num_features):
        super(LogisticRegressionModel, self).__init__()
        self.num_items = num_items
        self.num_features = num_features
        self.linears = nn.ModuleList([nn.Linear(num_features, 1) for _ in range(num_items)])

    def forward(self, x):
        outputs = []
        for i in range(self.num_items):
            outputs.append(torch.sigmoid(self.linears[i](x[:, i, :])))
        return torch.cat(outputs, dim=1)


Number of items: 10
Number of features: 20


In [14]:
def calRegret(optmodel, x, true_c, pred_r, true_r, true_obj, alpha):
    model = optmodel(x,pred_r, true_c, Q, alpha)
    sol, _ = model.solve()
    val = []
    for i in range(x.shape[1]):
        temp = np.sum((true_r[i] * sol[i]) ** (1 - alpha)) / (1 - alpha)
        val.append(temp)
    val = torch.tensor(np.array(val)).to(device)
    regret_loss = 0
    for i in range(x.shape[1]):
        regret_loss += true_obj[i] - val[i]
    return regret_loss

In [15]:
# Define the model
model = LogisticRegressionModel(num_items, num_features).to(device)

# Train-test split
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

batch_size = 32
loader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
loader_test = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.05)


In [16]:
import time

loss_log = []
num_epochs = 20
total_time = 0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    start_time = time.time()
    
    for x_batch, r_batch, c_batch, opt_sols_batch, opt_vals_batch in loader_train:
        
        x_batch = x_batch.float().to(device)
        r_batch = r_batch.float().to(device)
        c_batch = c_batch.float().to(device)
        opt_vals_batch = opt_vals_batch.float().to(device)

        optimizer.zero_grad()
        with torch.no_grad():
            model.eval()
            pred_r = model(x_batch).to("cpu").detach().numpy()
        model.train()

        regret_loss = calRegret(optModel, x_batch.detach().cpu().numpy(), c_batch.detach().cpu().numpy(), pred_r, r_batch.detach().cpu().numpy(), opt_vals_batch.detach().cpu().numpy(), alpha)
        loss_log.append(regret_loss.detach().cpu().numpy())
        regret_loss.requires_grad = True
        loss = regret_loss
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    end_time = time.time()
    epoch_time = end_time - start_time
    total_time += epoch_time

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss/len(loader_train):.4f}, Time: {epoch_time:.2f} seconds')

average_time = total_time / num_epochs
print(f'Average time per epoch: {average_time:.2f} seconds')


LogisticRegressionModel(
  (linears): ModuleList(
    (0-9): 10 x Linear(in_features=20, out_features=1, bias=True)
  )
)

Epoch [1/20], Loss: 5.9829, Time: 5.21 seconds


LogisticRegressionModel(
  (linears): ModuleList(
    (0-9): 10 x Linear(in_features=20, out_features=1, bias=True)
  )
)

Epoch [2/20], Loss: 5.9356, Time: 5.18 seconds


LogisticRegressionModel(
  (linears): ModuleList(
    (0-9): 10 x Linear(in_features=20, out_features=1, bias=True)
  )
)

...

Epoch [6/20], Loss: 5.9794, Time: 5.29 seconds


LogisticRegressionModel(
  (linears): ModuleList(
    (0-9): 10 x Linear(in_features=20, out_features=1, bias=True)
  )
)

Epoch [7/20], Loss: 5.8387, Time: 5.43 seconds


LogisticRegressionModel(
  (linears): ModuleList(
    (0-9): 10 x Linear(in_features=20, out_features=1, bias=True)
  )
)

KeyboardInterrupt: 

In [ ]:
demo = [i.tolist() for i in loss_log]
demo

[4.470971254232026,
 6.253414408137411,
 3.8935260356906873,
 6.498943766432147,
 4.891093938310952,
 7.357825899939346,
 6.310745932094596,
 5.974428944175525,
 5.836165131583201,
 6.036031089472701,
 6.371932788066573,
 5.815670852821821,
 5.302628063239279,
 6.815153269219461,
 5.2577811307324325,
 6.247972097209928,
 6.5235526635166075,
 5.043136293015888,
 6.208393732026954,
 5.444309903978482,
 6.589620457080075,
 6.344759942165673,
 4.169085566965894,
 5.8418538027017775,
 5.810741535543551,
 6.754523773001793,
 5.4387167433155525,
 6.052933588500961,
 5.48775395783176,
 5.502638415606498,
 6.738109818405286,
 4.8225387398826385,
 5.312491039840143,
 6.9288674280181155,
 6.296307722805739,
 5.689088394563553,
 4.515608725313047,
 6.319262309308609,
 6.13336718205743,
 4.209573468799022,
 6.618216962026821,
 5.546156666944176,
 8.005099778065837,
 6.4688995236131035,
 6.357097649071841,
 5.384003341605375,
 4.933188079827325,
 5.434968191879406,
 6.046880221725141,
 4.45031019262

In [ ]:
# Plot the loss curve
plt.plot(demo)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Curve')
plt.show()

Text(0.5, 0, 'Epoch')

Text(0, 0.5, 'Loss')

Text(0.5, 1.0, 'Training Loss Curve')

: 

## Regret

In [ ]:
def regret(predmodel, optmodel, dataloader, closed=False, alpha=0.5):
    """
    A function to evaluate model performance with normalized true regret

    Args:
        predmodel (nn): a regression neural network for cost prediction
        optmodel (optModel): an PyEPO optimization model
        dataloader (DataLoader): Torch dataloader from optDataSet

    Returns:
        float: true regret loss
    """
    # eval
    predmodel.eval()
    loss = 0
    optsum = 0

    if not closed:
    # load data
        for data in dataloader:
            x,r,c,opt_sol,opt_obj,_,_ = data
            # cuda
            x,r,c,opt_sol,opt_obj = x.cuda(),r.cuda(),c.cuda(),opt_sol.cuda(),opt_obj.cuda()
            # predict
            with torch.no_grad():
                pred_r = predmodel(x).to('cpu').detach().numpy()
            # solve
            for j in range(pred_r.shape[0]):
                loss += calRegret(optmodel, c[j], pred_r[j], r[j].to("cpu").detach().numpy(), opt_obj[j].item(), alpha)

                optsum += abs(opt_obj[j].item())
    # turn back to train mode
    predmodel.train()

    # normalize
    return loss / (optsum+1e-7)